In [1]:
import pyspark
from pyspark.sql import SparkSession

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-02.csv')

In [5]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [6]:
import pandas as pd

In [7]:
df_pandas = pd.read_csv('fhvhv_tripdata_2021-02.csv')

In [10]:
df_pandas = df_pandas.head(10)

In [11]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [12]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

Integer - 4 bytes
Long - 8 bytes

In [13]:
from pyspark.sql import types

In [52]:
schema = types.StructType([
                types.StructField('hvfhs_license_num', types.StringType(), True),
                types.StructField('dispatching_base_num', types.StringType(), True),
                types.StructField('pickup_datetime', types.TimestampType(), True),
                types.StructField('dropoff_datetime', types.TimestampType(), True),
                types.StructField('PULocationID', types.IntegerType(), True),
                types.StructField('DOLocationID', types.IntegerType(), True),
                types.StructField('SR_Flag', types.StringType(), True)
                ]
          )

In [53]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [54]:
df = df.repartition(24)

In [55]:
df.write.parquet('fhvhv/2021/02/', mode='overwrite')

In [56]:
df = spark.read.parquet('fhvhv/2021/02')

In [57]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [58]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.pickup_datetime >= '2021-02-15') \
  .filter(df.pickup_datetime < '2021-02-16').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-02-15 04:40:44|2021-02-15 04:59:52|          37|          76|
|2021-02-15 01:16:18|2021-02-15 01:27:37|          79|         170|
|2021-02-15 03:18:14|2021-02-15 03:27:04|          72|          71|
|2021-02-15 12:04:08|2021-02-15 12:29:07|          97|          37|
|2021-02-15 09:01:44|2021-02-15 09:12:20|         195|          33|
|2021-02-15 11:23:35|2021-02-15 11:36:23|          41|          42|
|2021-02-15 10:11:29|2021-02-15 10:21:18|          26|          26|
|2021-02-15 08:27:23|2021-02-15 08:45:18|         132|         138|
|2021-02-15 07:22:54|2021-02-15 07:39:33|          17|          25|
|2021-02-15 09:12:51|2021-02-15 09:19:23|         137|         233|
|2021-02-15 00:56:52|2021-02-15 01:23:28|           7|         265|
|2021-02-15 11:48:38|2021-02-15 12:24:18|       

In [59]:
df.registerTempTable('trips_data')

In [60]:
spark.sql("""
SELECT
    count(*) cnt
FROM
    trips_data
WHERE pickup_datetime >= '2021-02-15' and pickup_datetime < '2021-02-16'
""").show()

+------+
|   cnt|
+------+
|367170|
+------+



In [83]:
spark.sql("""
SELECT
    pickup_datetime, cast(dropoff_datetime as long) - cast(pickup_datetime as long) dur
FROM
    trips_data
order by dur desc
""").show()

+-------------------+-----+
|    pickup_datetime|  dur|
+-------------------+-----+
|2021-02-11 13:40:44|75540|
|2021-02-17 15:54:53|57221|
|2021-02-20 12:08:15|44039|
|2021-02-03 20:24:25|40653|
|2021-02-19 23:17:44|37577|
|2021-02-25 17:13:35|35010|
|2021-02-20 01:36:13|34806|
|2021-02-18 15:24:19|34612|
|2021-02-18 01:31:20|34555|
|2021-02-10 20:51:39|34169|
|2021-02-10 01:56:17|32476|
|2021-02-25 09:18:18|32439|
|2021-02-21 19:59:13|32223|
|2021-02-09 18:36:13|32087|
|2021-02-06 09:48:09|31447|
|2021-02-02 09:42:30|30913|
|2021-02-10 10:12:08|30856|
|2021-02-09 13:30:13|30732|
|2021-02-21 22:50:52|30660|
|2021-02-05 21:32:33|30511|
+-------------------+-----+
only showing top 20 rows



In [84]:
spark.sql("""
SELECT
    dispatching_base_num, count(*) cnt
FROM
    trips_data
group by dispatching_base_num
order by cnt desc
""").show()

+--------------------+-------+
|dispatching_base_num|    cnt|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



In [85]:
spark.sql("""
SELECT
    PULocationID, DOLocationID, count(*) cnt
FROM
    trips_data
group by PULocationID, DOLocationID
order by cnt desc
""").show()

+------------+------------+-----+
|PULocationID|DOLocationID|  cnt|
+------------+------------+-----+
|          76|          76|45041|
|          26|          26|37329|
|          39|          39|28026|
|          61|          61|25976|
|          14|          14|17934|
|         129|         129|14688|
|           7|           7|14688|
|          42|          42|14481|
|          37|          37|14424|
|          89|          89|13976|
|         216|         216|13716|
|          35|          35|12829|
|         132|         265|12542|
|         188|          61|11814|
|          95|          95|11548|
|          36|          37|11491|
|          37|          36|11487|
|          61|         188|11462|
|          61|         225|11342|
|         188|         188|11308|
+------------+------------+-----+
only showing top 20 rows

